# Welcome. 
This is a program that gives you visual matches between galaxies found in SDSS data and spectral lines fit in the GUI

In [36]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pandas.tools.plotting import table
import json
from IPython.display import display
import re
from astropy.cosmology import WMAP7 as cosmo
from astropy import units as u
from astropy import constants as const
import matplotlib.pyplot as plt
from astropy.table import QTable
import sympy as sp
import os
from os import listdir 
from astropy.table import Table, Column
from matplotlib.backends.backend_pdf import PdfPages
import astropy.io.fits as fits
import lam_list
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [37]:
import os

# Please type your path below. The following cell requires your attention

In [38]:
your_path = "C:/Users/justin/AstrResearch/bin/quasarfile/finalquasar"  #place your own path to the quasar file!!

for file in os.listdir(your_path):
    if file.endswith(".json"):
        print(file)
print('\x1b[1;35m'+ "copy from above and paste your json file below" + '\x1b[0m')
name = input('type file name here:  ') 
name_path = your_path + "/" + name
with open(name_path) as data_file:    
    data = json.load(data_file)

J1016+4706_werksquad.json
J1059+1441_werksquad.json
J1059+2517_werksquad.json
J1122+5755_werksquad.json
J1134+2555_werksquad.json
J1241+5721_werksquad.json
copy from above and paste your json file below
type file name here:  J1122+5755_werksquad.json


## Section 1: Finding Your Quasar

This section runs through a list of quasars and matches the quasar based on the name of your quasar file from the spectral analysis GUI. The assumption made is that your quasar file has the the quasar name in the file. 

Eg. "J1241+5721_formal.json"

If not, there will be a line for you to place the name of the quasar (Eg. "JXXXX+XXXX")

In [39]:
raw_quasar_radec = open("qsoinfo.txt", "r").read()  # this line requires that you downloaded qsoinfo.txt

In [40]:
revised_quasar_radec = raw_quasar_radec.splitlines(0)[1:]
qs0 = np.array([])
qso_long = np.array([])
gmag = np.array([])
ra = np.array([])
dec = np.array([])
qso_redshift = np.array([])

for i in range(0 , len(revised_quasar_radec)):
    line = revised_quasar_radec[i].split(" ")
    useful_list = np.array([])
    for t in range(0 , len(line)):
        if line[t] != "":
            useful_list = np.append(useful_list , line[t])    
    qs0 = np.append(qs0 , useful_list[0])
    qso_long = np.append(qso_long , useful_list[1])
    ra = np.append(ra , useful_list[2])
    dec = np.append(dec , useful_list[3])
    qso_redshift = np.append(qso_redshift , useful_list[4])
    gmag = np.append(gmag , useful_list[5])
    
qsolist = Table([qs0,qso_long,gmag,ra,dec, qso_redshift], 
      names=("qso","qso_long","gmag","ra","dec" , "qso_redshift"))

### WARNING: 
If your quasar file name doesn't have the name in the title or is not in the qsolist, the following cell will not work. Skip it and manually input the quasar values in the next cell

In [41]:
name_edit = name.split("_")
for i in range(0 , len(name_edit)):
    n = name_edit[i]
    if "J" == n[0] and (n[1] and n[2] and n[5] and n[9]) in "0123456789+-":
        name_compare = name_edit[i]
        break
    else:
        name_compare = "Failure, type in quasar data in cell below"


for i in range(0 , len(qsolist["qso"])):
    if name_compare == qsolist["qso"][i]:
        qso_index = qsolist[i]

print(qso_index)

   qso           qso_long       gmag    ra      dec   qso_redshift
---------- ------------------- ----- -------- ------- ------------
J1122+5755 J112244.89+575543.0 17.52 170.6870 57.9286        0.906


## Section 2: Using the Query

This section looks at the sdss query and puts the data of galaxies near the ra , dec value of the quasar.

In [42]:
ra_init = float(qso_index["ra"])
dec_init = float(qso_index["dec"])
redshift_init = float(qso_index["qso_redshift"])

# unhashtag the below lines if your quasar is not in qsolist, type in the exact quasar's (ra , dec , redshift) respectively
# ra_init =
# dec_init =
# redshift_init =

In [43]:
quasar_ra = ra_init * u.deg
quasar_dec = dec_init * u.deg
quasar_redshift = redshift_init

quasar_ra_1d = quasar_ra / u.deg
quasar_dec_1d = quasar_dec / u.deg

#Note, these are the parameters for searching los 
separation = .010 #of .010 redshift between an SDSS object and an absorber from the spectrum in the quasar
proximity = .5  # this is distance in Mpc from the LOS of the quasar.
spec_range = .3
zcc2_range = .05
ranges = zcc2_range    # set to spec_range if desired

ra_low = str(ra_init - ranges)
ra_high = str(ra_init + ranges)
dec_low = str(dec_init - ranges)
dec_high = str(dec_init + ranges)

In [44]:
import urllib.parse
import urllib.request

SEARCH_API_BASE = "http://cas.sdss.org/dr7/en/tools/search/x_sql.asp"

def run_query(sql):
    url = SEARCH_API_BASE + '?' + urllib.parse.urlencode({
        'format': 'csv',
        'cmd': sql,
    })
    with urllib.request.urlopen(url) as conn:
        return conn.read().decode('ascii')


quasar_data = run_query("""select  photoz.z, photoz.zErr, photoz2.photozcc2 , photoz2.photozerrcc2 ,
 photoz2.photozd1, u, err_u, ra, dec, extinction_r
from photoz , photoz2 , PhotoObjAll
where  PhotoObjAll.objID = photoz.objID AND
PhotoObjAll.objID = photoz2.objID AND
PhotoObjAll.ra between """ + ra_low + " and " + ra_high + """ AND
PhotoObjAll.dec between """ + dec_low + " and " + dec_high)


In [45]:
data_refined1 = quasar_data.splitlines(0)
z = np.array([])
zErr = np.array([])
photozcc2 = np.array([])
photozerrcc2 = np.array([])
photozd1 = np.array([])
u_xx = np.array([])
err_u = np.array([])
ra = np.array([])
dec = np.array([])
extinction_r = np.array([])
for i in range(1 , len(data_refined1)):
    line = data_refined1[i].split(",")
    z = np.append(z , float(line[0]))
    zErr = np.append(zErr , float(line[1]))
    photozcc2 = np.append(photozcc2 , float(line[2]))
    photozerrcc2 = np.append(photozerrcc2 , float(line[3]))
    photozd1 = np.append(photozd1 , float(line[4]))
    u_xx = np.append(u_xx , float(line[5]))
    err_u = np.append(err_u , float(line[6]))
    ra = np.append(ra , float(line[7]))
    dec = np.append(dec , float(line[8]))
    extinction_r = np.append(extinction_r , float(line[9]))
    
    
query_data = Table([z,zErr,photozcc2,photozerrcc2,photozd1,u_xx,err_u,ra,dec,extinction_r], 
      names=("z","zErr","photozcc2","photozerrcc2","photozd1","u","err_u","ra","dec","extinction_r"))


## Section 3: Analyzing the Query Data

This section is a series of functions applied to the data file from the SDSS query. The result is a new table with new data derived through functions from old data

In [46]:
arcsecperrad = (u.arcsec / u.rad).si
def real_angular_diam_dist(redshift):
    value = cosmo.kpc_proper_per_arcmin(redshift)
    return(value.to(u.kpc / u.arcsec))
    

def angular_separation(ra1,dec1,ra2,dec2):
    dela = ra1 * u.deg- ra2 
    deldec = dec1 * u.deg - dec2
    delthet = np.sqrt((dela * np.cos(deldec))**2 + deldec ** 2)
    return(delthet.to(u.arcsec))
    
def angular_separation_notdegrees(ra1h, ra1m, ra1s, dec1d, dec1m, dec1s, ra2h, ra2m, ra2s, dec2d, dec2m, dec2s):
    dela = ra1h * u.hourangle + ra1m * u.arcmin + ra1s * u.arcsec - (ra2h * u.hourangle + ra2m * u.arcmin + ra2s * u.arcsec)
    deldec = dec1d * u.deg + dec1m * u.arcmin + dec1s * u.arcsec - (dec2d * u.deg + dec2m * u.arcmin + dec2s * u.arcsec)
    print(deldec)
    deladeg = dela.to(u.deg)
    deldecdeg = deldec.to(u.deg)
    delthet = np.sqrt((deladeg*np.cos(deldecdeg))**2 + deldecdeg ** 2)
    print(delthet)

def distance_from_los(ang_sep, ang_diam_dist):
    final = ((ang_sep/u.arcsec) * (ang_diam_dist* u.arcsec/u.kpc)) 
    final = final/1000
    return final
    


In [47]:
zcc2redshift = query_data['photozcc2', 'ra', 'dec']

In [48]:
zcc2redshift['angdiam'] = real_angular_diam_dist(zcc2redshift['photozcc2'])

zcc2redshift['angsep'] = angular_separation(zcc2redshift['ra'],zcc2redshift['dec'],quasar_ra,quasar_dec)
zcc2redshift['dist_from_los'] = distance_from_los(zcc2redshift['angsep'], zcc2redshift['angdiam'])
zcc2redshift["dist_from_los"].unit = u.Mpc
condensed_table = zcc2redshift['photozcc2', 'ra', 'dec', 'angsep', 'dist_from_los']


condensed_table.sort('photozcc2')
# condensed_table

## Section 4: Analyzing the JSON Spectrum file

This takes all the data in the JSON file and and puts the fittings into various lists such as: graphponents_filter as well as 
(strong_redshift , weak_redshift , all_redshift , metal_abs). 

In [49]:
radshift = []
ra = []
dec= []

In [50]:
name_list = ['HI', 'HeI', 'AlII', 'AlIII', 'ArVI', 'CI', 'CII', 'CII*', 'CIII', 'CIV', 'CaII', 'CaVIII',
             'CaX', 'ClII', 'FeII', 'FeIII', 'MgI', 'MgII', 'MgX', 'NI', 'NII', 'NIII',
             'NIV', 'NV', 'NaI', 'NeII', 'NeIII', 'NeIV', 'NeVI', 'NeVII', 'NeVIII', 'NiII', 'OI', 
             'OII', 'OIII', 'OIV', 'OV', 'OVI', 'OVII', 'OVIII', 'PII', 'PIII', 'PV', 'SII', 'SIII', 
             'SIV', 'SV', 'SVI', 'SiII', 'SiII*', 'SiIII', 'SiIV', 'TiII', 'ZnII']

components = []
graphponents = []
systems=['z0.00000_MW']
for cmp in data["cmps"]:
    cmp_dict=data["cmps"][str(cmp)]
    cmp_data=''
    if cmp[-2:]=="HI":
        systems.append(str(cmp))
    cmp_data+=str(cmp)+';'
    cmp_data+=str(cmp_dict["Reliability"])+';'
    if cmp_dict["Comment"]=='':
        systems.append(str(cmp))
        cmp_data+='None;'
    else:
        cmp_data+=str(cmp_dict["Comment"])+';'
    cmp_data+=str(cmp_dict["Nfit"])+';'
    cmp_data+=str(cmp_dict["bfit"])+';'
    components.append(cmp_data)
    graphponents.append(str(cmp))


comp_broken =[]
for x in range(0 , len(components)):
    comp_broken.append(components[x].split(";"))

graphponents_filter = []   
for filter_ in name_list: 
    graphponents_filter.append(len([pos for pos, s in enumerate(graphponents) if filter_ in s]))


In [51]:
name_list_full = np.array(['NeVI', 'NeII', 'OII', 'OII', 'OII', 'OII', 'NeVI', 'NeII', 'NII',
       'NeIII', 'OIII', 'HeI', 'HeI', 'HeI', 'HeI', 'NII', 'HeI', 'OII',
       'OII', 'NeIV', 'NeIV', 'CII', 'NeIV', 'ArVI', 'OIV', 'OIV', 'CaX',
       'NeVI', 'CII', 'NeVI*', 'NeVI*', 'NeV', 'NeV*', 'NeV*', 'CaX',
       'CaVIII', 'HeI', 'ArVII', 'CII', 'OIV', 'MgX', 'MgX', 'OV', 'SII',
       'SII', 'NII', 'SIV', 'SIII', 'SIII', 'NaIX', 'NIII', 'NIII', 'CII',
       'NaIX', 'SIII', 'OIII', 'SIII', 'SIV', 'SIV', 'NIII', 'SII', 'SII',
       'NIV', 'SII', 'NeVIII', 'NeVIII', 'SV', 'OIV', 'OI', 'SIV', 'OII',
       'OIII', 'OII', 'OII', 'FeIII', 'CII', 'FeIII', 'OI', 'OI', 'ClV',
       'SiII', 'CII', 'CII', 'SII', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI',
       'HI', 'HI', 'HI', 'HI', 'HI', 'PIII', 'HI', 'HI', 'HI', 'HI', 'HI',
       'NII', 'HI', 'HI', 'DI', 'PIII', 'HI', 'HI', 'HI', 'HI', 'OI', 'OI',
       'HI', 'OI', 'OI', 'DI', 'HI', 'OI', 'OI', 'HI', 'SVI', 'OI', 'HI',
       'SVI', 'CI', 'CI*', 'CI**', 'OI', 'HI', 'OI', 'PII', 'NI', 'NI',
       'NI', 'OI', 'DI', 'HI', 'OI', 'CIII', 'OI', 'OI', 'OI', 'NIII',
       'SiII', 'PIII', 'AsII', 'SIII', 'SiII', 'HI', 'OI', 'MgII', 'MgII',
       'OVI', 'CII', 'CII*', 'OVI', 'OI', 'ArI', 'FeII', 'FeII', 'SIV',
       'FeII', 'ClII', 'FeII', 'ArI', 'ClII', 'FeII', 'NII', 'FeII',
       'FeII', 'FeII', 'CI', 'CI', 'CI', 'CI*', 'CI*', 'CI*', 'CI*',
       'CI**', 'CI**', 'CI**', 'CI**', 'CI**', 'CI', 'PV', 'CI*', 'CI**',
       'CI**', 'CI', 'CI*', 'CI*', 'CI*', 'FeII', 'CI**', 'CI*', 'CI**',
       'CI**', 'CI**', 'CI', 'FeIII', 'CI*', 'CI*', 'CI**', 'CI**', 'CI**',
       'FeII', 'PV', 'CI', 'CI*', 'CI*', 'CI*', 'CI', 'CI**', 'CI**',
       'CI*', 'CI*', 'CI**', 'CI**', 'CI', 'CI*', 'CI*', 'CI**', 'CI**',
       'CI**', 'FeII', 'NI', 'NI', 'NI', 'CI', 'CI*', 'CI*', 'CI*', 'CI**',
       'CI**', 'CI**', 'CI*', 'FeII', 'FeII', 'FeII', 'PII', 'PII', 'CI**',
       'CI', 'KrI', 'CI*', 'SIII', 'SiII', 'CI**', 'CI**', 'CI', 'SiII',
       'CI*', 'MnII', 'SiII*', 'CI*', 'MnII', 'NI', 'NI', 'NI', 'SiIII',
       'DI', 'HI', 'KrI', 'NV', 'MgII', 'MgII', 'NV', 'SII', 'SII', 'SII',
       'SiII', 'FeII', 'CI', 'CI*', 'CI*', 'CI*', 'CI**', 'CI**', 'AsII',
       'SiII*', 'SiII*', 'CI', 'CI*', 'CI**', 'CI**', 'CI', 'CI*', 'CI**',
       'CI', 'CI*', 'CI*', 'CI**', 'CI**', 'CI**', 'CI**', 'CI**', 'CI**',
       'OI', 'SiII', 'OI*', 'OI**', 'SiII*', 'SI', 'NiII', 'TlII', 'CI',
       'CI*', 'CI*', 'CI*', 'CI**', 'CI**', 'CII', 'CII*', 'ClI', 'OI',
       'AsII', 'CuII', 'BII', 'CuII', 'NiII', 'SiIV', 'SnII', 'SiIV',
       'GaII', 'SI', 'PbII', 'NiII', 'CoII', 'NiII', 'NiII', 'XeI', 'SiII',
       'PII', 'SiII*', 'CIV', 'CIV', 'CI', 'CI*', 'CoII', 'SiI', 'GeII',
       'FeII', 'FeII', 'CI', 'CI*', 'CI**', 'FeII*', 'FeII*', 'FeII*',
       'FeII*', 'FeII*', 'FeII*', 'FeII*', 'CI*', 'CI', 'CI**', 'CI*',
       'CI*', 'CI**', 'AlII', 'PbII', 'MgI', 'NiII', 'MgI', 'NiII', 'NiII',
       'MgI', 'NiII', 'NiII', 'NiII', 'NiII', 'ZrIII', 'NiII', 'SI',
       'SiII', 'SiII*', 'MgI', 'AlIII', 'AlIII', 'SiIII', 'FeII', 'TiII',
       'TiII', 'CoII', 'CoII', 'ZnII', 'CrII', 'MgI', 'CrII', 'CrII',
       'ZnII', 'CrII', 'VII', 'CdII', 'FeII', 'FeII', 'SiII', 'FeII',
       'FeII', 'FeII', 'FeII*', 'FeII*', 'FeII*', 'FeII*', 'FeII*',
       'FeII*', 'FeII*', 'FeII*', 'FeII*', 'MnII', 'FeII', 'MnII', 'FeII*',
       'FeII', 'MnII', 'FeII*', 'FeII*', 'FeII*', 'FeII*', 'FeII*',
       'FeII*', 'FeII*', 'FeII*', 'FeII*', 'FeII*', 'VII', 'MgII', 'MgII',
       'MgI', 'TiII', 'BeII', 'BeII', 'TiII', 'TiII', 'TiII', 'CaII',
       'CaII', 'NaI', 'NaI', 'LiI'])

lam_list_full = np.array([  401.14,   405.85,   418.6 ,   429.92,   430.04,   430.18,
         433.18,   446.26,   475.65,   489.5 ,   507.39,   507.72,
         508.64,   510.  ,   522.21,   533.51,   537.03,   539.09,
         539.55,   541.13,   542.07,   543.26,   543.89,   551.36,
         553.33,   554.08,   557.76,   558.59,   560.24,   562.71,
         562.8 ,   568.42,   569.76,   569.83,   574.01,   582.85,
         584.33,   585.75,   594.8 ,   608.4 ,   609.79,   624.95,
         629.73,   640.9 ,   641.77,   644.63,   657.34,   677.75,
         681.47,   681.72,   685.  ,   685.51,   687.05,   694.27,
         698.73,   702.33,   724.29,   744.91,   748.4 ,   763.34,
         763.66,   764.42,   765.15,   765.69,   770.41,   780.32,
         786.48,   787.71,   791.97,   809.67,   832.76,   832.93,
         833.33,   834.47,   844.29,   858.09,   859.72,   877.8 ,
         877.88,   883.12,   889.72,   903.62,   903.96,   906.88,
         912.7 ,   912.77,   912.84,   912.92,   913.01,   913.1 ,
         913.21,   913.34,   913.48,   913.64,   913.83,   913.97,
         914.04,   914.29,   914.58,   914.92,   915.33,   915.61,
         915.82,   916.43,   916.93,   917.12,   917.18,   918.13,
         919.35,   920.96,   921.86,   922.2 ,   923.15,   924.95,
         925.44,   925.97,   926.23,   929.52,   930.26,   930.75,
         933.38,   936.63,   937.8 ,   944.52,   945.19,   945.34,
         945.58,   948.69,   949.74,   950.88,   963.8 ,   963.99,
         964.63,   965.04,   971.74,   972.27,   972.54,   976.45,
         977.02,   988.58,   988.65,   988.77,   989.8 ,   989.87,
         998.  ,  1009.43,  1012.5 ,  1020.7 ,  1025.72,  1025.76,
        1025.97,  1026.11,  1031.93,  1036.34,  1037.02,  1037.62,
        1039.23,  1048.22,  1055.26,  1062.15,  1062.66,  1063.18,
        1063.83,  1063.97,  1066.66,  1071.04,  1081.87,  1083.99,
        1096.88,  1106.36,  1112.05,  1112.27,  1117.  ,  1117.13,
        1117.2 ,  1117.24,  1117.34,  1117.39,  1117.54,  1117.58,
        1117.6 ,  1117.67,  1117.73,  1117.87,  1117.98,  1118.07,
        1118.41,  1118.49,  1121.45,  1121.64,  1121.66,  1121.76,
        1121.97,  1122.  ,  1122.  ,  1122.1 ,  1122.33,  1122.34,
        1122.44,  1122.52,  1122.64,  1122.8 ,  1122.98,  1123.11,
        1123.13,  1125.45,  1128.01,  1128.08,  1128.25,  1128.29,
        1128.41,  1128.48,  1128.63,  1128.75,  1128.82,  1129.08,
        1129.14,  1129.16,  1129.19,  1129.4 ,  1129.62,  1129.75,
        1129.92,  1129.97,  1133.67,  1134.17,  1134.41,  1134.98,
        1138.38,  1138.56,  1138.6 ,  1138.74,  1138.95,  1139.09,
        1139.12,  1140.22,  1142.37,  1143.23,  1144.94,  1149.96,
        1152.82,  1157.33,  1158.32,  1164.87,  1189.07,  1190.2 ,
        1190.42,  1190.64,  1191.84,  1193.03,  1193.29,  1194.3 ,
        1197.18,  1197.39,  1197.88,  1199.39,  1199.55,  1200.22,
        1200.71,  1206.5 ,  1215.34,  1215.67,  1235.84,  1238.82,
        1239.93,  1240.39,  1242.8 ,  1250.58,  1253.81,  1259.52,
        1260.42,  1260.53,  1260.74,  1260.93,  1261.  ,  1261.12,
        1261.43,  1261.55,  1263.77,  1264.74,  1265.  ,  1270.14,
        1270.41,  1270.84,  1274.11,  1276.48,  1276.75,  1277.19,
        1277.25,  1277.28,  1277.51,  1277.55,  1277.72,  1277.95,
        1279.23,  1279.5 ,  1280.33,  1302.17,  1304.37,  1304.86,
        1306.03,  1309.28,  1316.54,  1317.22,  1321.64,  1328.83,
        1329.08,  1329.1 ,  1329.12,  1329.58,  1329.6 ,  1334.53,
        1335.71,  1347.24,  1355.6 ,  1355.93,  1358.77,  1362.46,
        1367.95,  1370.13,  1393.76,  1400.44,  1402.77,  1414.4 ,
        1425.03,  1433.91,  1454.84,  1466.21,  1467.26,  1467.76,
        1469.61,  1526.71,  1532.53,  1533.43,  1548.19,  1550.77,
        1560.31,  1560.68,  1574.55,  1589.17,  1602.49,  1608.45,
        1611.2 ,  1613.38,  1613.8 ,  1614.51,  1618.47,  1621.69,
        1629.16,  1631.13,  1634.35,  1636.33,  1639.4 ,  1656.27,
        1656.93,  1657.01,  1657.38,  1657.91,  1658.12,  1670.79,
        1682.13,  1683.41,  1703.41,  1707.06,  1709.6 ,  1741.55,
        1747.79,  1751.92,  1772.28,  1773.95,  1783.32,  1798.05,
        1804.47,  1807.31,  1808.01,  1816.93,  1827.94,  1854.72,
        1862.79,  1892.03,  1901.77,  1910.61,  1910.95,  1941.29,
        2012.17,  2026.14,  2026.27,  2026.48,  2056.26,  2062.23,
        2062.66,  2066.16,  2138.83,  2145.07,  2249.88,  2260.78,
        2335.12,  2344.21,  2374.46,  2382.77,  2383.79,  2389.36,
        2396.15,  2396.36,  2399.97,  2405.62,  2407.39,  2411.25,
        2414.05,  2576.88,  2586.65,  2594.5 ,  2599.15,  2600.17,
        2606.46,  2607.87,  2612.65,  2614.61,  2618.4 ,  2621.19,
        2622.45,  2626.45,  2629.08,  2631.83,  2632.11,  2683.89,
        2796.35,  2803.53,  2852.96,  3073.88,  3131.33,  3131.97,
        3230.13,  3242.93,  3384.74,  3934.78,  3969.59,  5891.58,
        5897.56,  6709.61])

In [52]:
#list of element and z value

In [53]:
raw_spec_data = []
for i in range(len(comp_broken)):
    title = comp_broken[i][0].split("_")
    element = title[1]
    z_value = float(title[0][1:])
    for j in range(len(lam_list_full)):
        if element == name_list_full[j]:
            lam_emit = lam_list_full[j] * (z_value + 1)
    dummy = [lam_emit , element]
    raw_spec_data.append(dummy)

In [54]:
raw_spec_data = sorted(raw_spec_data)

In [55]:
raw_spec_lam = []
raw_spec_elem = []

for i in range(len(raw_spec_data)):
    raw_spec_lam.append(raw_spec_data[i][0])
    raw_spec_elem.append(raw_spec_data[i][1])

In [56]:
new_data = []
name = []
rating = []
comment = []
Nc = []
bc = []
raw_z = []
strong_abs = []
strong_redshift =[]
has_metals = []
weak_abs = []
weak_redshift = []
all_redshift = []
all_abs = []
metal_abs = []

for component in components:
    cmp_info = component.split(';')
    #print(cmp_info)
    if component[1]=='-':
        z_simple = component[2:8]
    else:
        z_simple = component[1:7]
    condition1 , condition2 , condition3 , condition4 = True , True , True , True
    
    
    # condition 1 , what element do you want to find?
    if "HI" in cmp_info[0]:
        condition1 = True
    else:
        
        condition1 = False

    if float(cmp_info[3]) > 10:
        condition2 = True
    else:
        condition2 = False
    
    if "" in cmp_info[2]:
        condition3 = True
    else:
        condition3 = False
        
    conditions = condition1 and condition2 and condition3 and condition4
   
    if conditions == True:
        raw_z.append(float(z_simple))
        name.append(cmp_info[0])
        rating.append(cmp_info[1])
        comment.append("{0:.60s}".format(cmp_info[2]))    #change 60s to larger number for longer comments
        Nc.append(float("{0:.6s}".format(cmp_info[3])))
        bc.append(float("{0:.6s}".format(cmp_info[4])))

    if float(cmp_info[3]) > 13.8:
        condition5 = True
    else:
        condition5 = False

    if float(cmp_info[3]) > 13.0 and float(cmp_info[3]) <= 13.8  :
        condition6 = True
    else:
        condition6 = False
  
    if float(cmp_info[3]) > 10.0 :
        condition7 = True
    else:
        condition7 = False    
    
    if condition5 and condition1== True:
        strong_abs.append(float("{0:.6s}".format(cmp_info[3])))
        strong_redshift.append(float(z_simple))
        
    if condition6 and condition1== True:
        weak_abs.append(float("{0:.6s}".format(cmp_info[3])))
        weak_redshift.append(float(z_simple))
        
    if condition7 and condition1== True:
        all_abs.append(float("{0:.6s}".format(cmp_info[3])))
        all_redshift.append(float(z_simple))
        
    if condition1 == False: 
        metal_abs.append(float(z_simple))

## Section 5: Creating and Outputting Graphs

This section compares data from the SDSS query (Section 3) with data from the JSON spectrum file (Section 4) to find matches between real galaxies and Lymann Alpha fits.

In [57]:
workingarray1 = []
workingarray2 = []
workingarray3 = []
workingarray4 = []
workingarray5 = []
quantity_less_comparison = condensed_table 
quantity_less_comparison['dist_from_los'] = quantity_less_comparison['dist_from_los'] / u.Mpc 

for Idx1,Val1 in enumerate(condensed_table['photozcc2']):
    for Idx2,Val2 in enumerate(strong_redshift):
        difference = abs(condensed_table['photozcc2'][Idx1] - strong_redshift[Idx2])
        if difference < separation and condensed_table['dist_from_los'][Idx1] < proximity: 
            workingarray1.append(condensed_table['photozcc2'][Idx1])
            workingarray2.append(float(quantity_less_comparison['dist_from_los'][Idx1]))
            workingarray3.append(strong_redshift[Idx2])
            workingarray4.append(condensed_table['ra'][Idx1])
            workingarray5.append(condensed_table['dec'][Idx1])

comparison_table= Table([workingarray1, workingarray2, workingarray3, \
workingarray4, workingarray5], names=('photozcc2', 'dist_from_los', \
'strong_redshift','ra','dec'), meta={'name': 'comparisontable'})

comparison_table["delta_z"] = comparison_table["photozcc2"] - comparison_table["strong_redshift"]
comparison_table = comparison_table["photozcc2" , "strong_redshift" , "delta_z" ,
                                    "dist_from_los" , "ra" , "dec"]
scale = len(comparison_table)

figtable , ax = plt.subplots(1,1)
figtable.set_size_inches(16, 8)
clust_data = comparison_table
collabel=('photoz2', 'dist_from_los', 'hir','delta_z','ra','dec')
table = ax.table(cellText=clust_data,colLabels=collabel,loc='top' , cellLoc='left', 
                 bbox=[-0.1, 0.2, 0.7, 1.1] , 
                 colWidths = [.9,1.6,.7,1,1.4,1.3])

ax.axis("off")
table.auto_set_font_size(False)
table.set_fontsize(14)
#ax.text(.5 , 1 , "Matches Data", horizontalalignment='center' , fontsize = 20)
ax.table?

#plt.show()

In [58]:
%%capture
bin_redshift = condensed_table['dist_from_los']
bin_sep = 0.150
bins = [0, bin_sep]
hist, bin_edges = np.histogram(bin_redshift, bins=bins)
bins = 15
hist, bin_edges = np.histogram(bin_redshift, bins=bins)


#######
figtable , ax = plt.subplots(1,1)
figtable.set_size_inches(16, 16)
ax.set_title("Matches list between galaxy and HI lines" , fontsize = 20)
clust_data = comparison_table
collabel=('phtzcc2', 'HI_z', 'delta_z', 'dist_from_los', 'ra','dec')
table = ax.table(cellText=clust_data,colLabels=collabel,loc='middle' , cellLoc='left', 
                 bbox=[0.12, 0.5 - 0.012 * scale, 0.7, 0.023 * scale] , 
                 colWidths = [1.1,.9,1.1,1.8,1.6,1.6])
ax.axis("off")
table.auto_set_font_size(False)
table.set_fontsize(12)
#ax.text(.5 , 1 , "Matches Data", horizontalalignment='center' , fontsize = 20)

########
fig1, ax = plt.subplots(1,1)
ax.set_title('number of absorbers. Data sorted in bins of '+str(bin_sep))
ax.set_xlabel('distance from LOS in MPC')
ax.set_ylabel('count')
ax.hist(bin_redshift, bins, edgecolor = 'black');

#######
fig2, ax = plt.subplots(1,1)
ax.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'], 'ro',color = 'blue');
ax.set_title('distance from los in MPC versus redshift')
ax.set_xlabel('redshift')
ax.set_ylabel('distance from los in MPC')

valer = 0
valery = 0

#######
fig3, walker = plt.subplots(1,1)
fig3.set_size_inches(15,2)
walker.set_xlim(0,quasar_redshift + .05)
walker.vlines(quasar_redshift, -1.8, -.2, color='green', linewidth=3)   
walker.vlines(strong_redshift, -1.8, -.2, color='red', linewidth=1)
walker.vlines(quasar_redshift, -1,1,color='none',linewidth=0)
walker.vlines(weak_redshift, -1.8, -.2, color='gray', linewidth=.5)
walker.vlines(metal_abs,-3,-2,color='black',linewidth = .5)
walker.set_title('Green = RED = strong absorbers, Blue = Objects in LOS from SDSS, Purple = Quasar, GRAY = weak absorbers, BLACK = metals')
walker.plot(condensed_table['photozcc2'], np.zeros_like(condensed_table['photozcc2']) + valer, '|', markersize=15,);


baler = 0
balery = 0

#########
fig4, two_walker = plt.subplots(1,1)
fig4.set_size_inches(15,15)
two_walker.set_xlim(0,quasar_redshift + .05)
two_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
two_walker.vlines(strong_redshift, 0,  .25, color='red', linewidth=1)
two_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
two_walker.vlines(weak_redshift,0,  .25, color='gray', linewidth=.5)
two_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
two_walker.set_title('Red = strong absorbers, Purple = strong absorbers with strong sdss fits, Blue = Objects in LOS from SDSS, Green= Quasar, GRAY = weak absorbers, BLACK = metals')
two_walker.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'],marker = 'o',linestyle='none',markersize = 6);



########
fig5, ax = plt.subplots(1,1)
ax.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'], 'ro',color = 'purple');
ax.set_title('distance from los in MPC versus redshift')
ax.set_xlabel('redshift')
ax.set_ylabel('distance from los in MPC')



#######
fig6, three_walker = plt.subplots(1,1)
fig6.set_size_inches(15,15)
three_walker.set_xlabel("Redshift")
three_walker.set_ylabel("Mpc")
three_walker.set_xlim(0,quasar_redshift + .05)
three_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
three_walker.vlines(comparison_table['strong_redshift'], -.1,  comparison_table['dist_from_los'] + .1, color='m', linewidth=2)
three_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
three_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
three_walker.set_title('Purple = strong absorbers with good fits || \n Green = Quasar, BLACK = metals')
three_walker.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'],marker = '*',linestyle='none'
                  ,markersize = 10,color = '#040273');


######
fig7, two_walker = plt.subplots(1,1)
fig7.set_size_inches(15,15)
two_walker.set_xlabel("Redshift")
two_walker.set_ylabel("Mpc")
two_walker.set_xlim(0,quasar_redshift + .05)
two_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
two_walker.vlines(strong_redshift, 0,  .25, color='red', linewidth=1)
two_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
two_walker.vlines(weak_redshift,0,  .25, color='gray', linewidth=.5)
two_walker.vlines(comparison_table['strong_redshift'], 0,  .25 , color='m', linewidth=2)
two_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
two_walker.set_title('RED = strong absorbers \n PURPLE = strong absorbers with strong sdss fits \n \
BLUE = Objects in LOS from SDSS \n GREEN= Quasar \n GRAY = weak absorbers \n BLACK = metals \n STAR = matching galaxy')
two_walker.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'],marker = 'o',linestyle='none',markersize = 6);
two_walker.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'],marker = '*',linestyle='none',
                markersize = 10, color = '#040273');


######
fig8, ax = plt.subplots(1,1)
ax.set_title('histogram of all absorbers based on redshift')
ax.hist(all_redshift, bins=10, facecolor='MediumOrchid');


######
fig9, ax = plt.subplots(1,1)
ax.set_title('historgram of strong absorbers based on redshift')
ax.hist(strong_redshift, bins=10, facecolor='MediumOrchid');


######
fig10, ax = plt.subplots(1,1)
ax.set_title('histogram of weak absorbers based on redshift')
ax.hist(weak_redshift, bins=10, facecolor='MediumOrchid');

######
fig11 , ax = plt.subplots(1,1)
fig11.set_size_inches(15,8)
ax.text(-2.8, 0.2, 'HI={0:d}'.format(graphponents_filter[0]), color='red', fontsize=18)
amt_ticks = np.arange(len(graphponents_filter))
ax.bar(amt_ticks,[0] + graphponents_filter[1:]);
ax.set_title("Quantity per Metal" , fontsize = 20)
ax.set_xticks(np.arange(len(graphponents_filter)))
ax.set_xticklabels((name_list) , rotation = "vertical")


valer = 0
valery = 0
#######
fig12, walker = plt.subplots(1,1)
fig12.set_size_inches(15,2)
walker.set_xlim(0,quasar_redshift + .05)
walker.vlines(quasar_redshift, -1, 1, color='green', linewidth=3)   
walker.vlines(strong_redshift,-1, 1, color='red', linewidth=1)
walker.vlines(quasar_redshift,-1, 1,color='none',linewidth=0)
walker.vlines(weak_redshift, -1, 1, color='gray', linewidth=.5)
walker.vlines(metal_abs,-2,-1,color='black',linewidth = .5)
plt.title('Green = RED = strong absorbers, Purple = Quasar, GRAY = weak absorbers, BLACK = metals')


note = """{0} matches were found.
""".format(len(comparison_table))

header = "Graphical Analysis of QSO {0:s}".format(name_compare)

left, width = .25, .5
bottom, height = .25, .5
right = left + width
top = bottom + height
fig_text , ax = plt.subplots(1,1)
fig_text.set_size_inches(10,13)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
ax.text(0.5*(left+right), (bottom+top), header,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=24, color='purple',
        transform=ax.transAxes)
ax.text(0.5*(left+right), 0.5*(bottom+top), note,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=12, color='black',
        transform=ax.transAxes)


pp = PdfPages(name_compare + '_Analysis2.pdf')
pp.savefig(fig_text)
pp.savefig(figtable)

pp.savefig(fig7) # main full graph with matches
pp.savefig(fig6) # main graph; matches only
pp.savefig(fig2) # MPC vs. redshift of galaxies
pp.savefig(fig5) # MPC vs. redshift of galaxy matches
pp.savefig(fig1) # number of absorbers
pp.savefig(fig8) # general histograms
pp.savefig(fig9) # general histograms
pp.savefig(fig10) # general histograms
pp.savefig(fig11) # quantity metal bar plot
pp.savefig(fig3) # quasar analysis only
#pp.savefig(fig12) # quasar analysis only
#pp.savefig(fig4) # main full graph without matches



pp.close()
    